In [1]:
# Import needed packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# If you're working in Jupyter Notebook, include the following so that plots will display:
%matplotlib inline

dataframe = pd.read_csv('student-mat.csv')
print(dataframe.head())

for c in dataframe.columns:
    print(c)
    print(dataframe[c].value_counts(dropna=False))
print(dataframe.columns)

from sklearn.model_selection import train_test_split

target = 'internet'
y = dataframe[target].map({'yes': 1, 'no': 0})

X = dataframe.drop(labels = [target], axis='columns')


# drop non-numberic fields
yes_no = ['activities','schoolsup','famsup','paid','nursery','higher','romantic']
# map yes/no to 1/0
for c in yes_no:
    X[c] = X[c].map({'yes': 1, 'no': 0})
# translate non_numeric fields to int
non_numeric = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob',
   'Fjob', 'reason', 'guardian']
for c in non_numeric:
    X[c] = X[c].astype('category').cat.codes

    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
# split validate set and test set
validate_train,holdout_train,validate_test,holdout_test = train_test_split(X_test, y_test, test_size=0.5)

baseline_classifier_model = RandomForestClassifier()
# Train the baseline model on training data
baseline_classifier_model = baseline_classifier_model.fit(X_train, y_train)

# Use the forest's predict method on the holdout
predictions = baseline_classifier_model.predict(holdout_train)

# Calculate the accuracy_score
accuracy_score(predictions, holdout_test)

  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  ...  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher  ...   
1     GP   F   17       U     GT3       T     1     1  at_home     other  ...   
2     GP   F   15       U     LE3       T     1     1  at_home     other  ...   
3     GP   F   15       U     GT3       T     4     2   health  services  ...   
4     GP   F   16       U     GT3       T     3     3    other     other  ...   

  famrel freetime  goout  Dalc  Walc health absences  G1  G2  G3  
0      4        3      4     1     1      3        6   5   6   6  
1      5        3      3     1     1      3        4   5   5   6  
2      4        3      2     2     3      3       10   7   8  10  
3      3        2      2     1     1      5        2  15  14  15  
4      4        3      2     1     2      5        4   6  10  10  

[5 rows x 33 columns]
school
GP    349
MS     46
Name: school, dtype: int64
sex
F    208
M    187
Name: sex, d

0.8

In [2]:
best_model = None
for i in range(5):
	classifier_model = RandomForestClassifier()
	classifier_model = classifier_model.fit(X_train, y_train)
	predictions = classifier_model.predict(validate_train)
	score = accuracy_score(validate_test, predictions)
	print("%d times running, scoring %f"%(i+1,score))
	if best_model is None or best_model[1] < score:
		best_model = (classifier_model, score)
	

model, score = best_model

1 times running, scoring 0.864407
2 times running, scoring 0.864407
3 times running, scoring 0.847458
4 times running, scoring 0.864407
5 times running, scoring 0.847458


In [2]:
# View confusion matrix for test data and predictions
confusion_matrix(holdout_test, predictions)
# Get and reshape confusion matrix data
matrix = confusion_matrix(holdout_test, predictions)
matrix = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

# Build the plot
plt.figure(figsize=(16,7))
sns.set(font_scale=1.4)
sns.heatmap(matrix, annot=True, annot_kws={'size':10},
            cmap=plt.cm.Greens, linewidths=0.2)

# Add labels to the plot
class_names = ['activities','schoolsup','famsup','paid','nursery','higher','romantic']
tick_marks = np.arange(len(class_names))
tick_marks2 = tick_marks + 0.5
plt.xticks(tick_marks, class_names, rotation=25)
plt.yticks(tick_marks2, class_names, rotation=0)
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix for Random Forest Model')
plt.show()

ValueError: Found input variables with inconsistent numbers of samples: [60, 59]

In [6]:
# View the classification report for test data and predictions
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        17
           1       0.83      0.99      0.90        82

    accuracy                           0.82        99
   macro avg       0.41      0.49      0.45        99
weighted avg       0.68      0.82      0.75        99

